In [ ]:
#In this file I will generate all negatives. I will start by generating all possible negative sequences.
#Then, I will take the sequences in the training set and in the test test and create a set of negatives to remove from the set of all negatives.
#This step just lightweights negatives, the full redundancy filtering is performed during negative selection.
#I have not added all remaining negatives in pickles since it is a big file and I do not want to store it in the repository, so this script needs to be run
#before hard negative mining
import pickle
import numpy as np
from pathlib import Path
import copy
import random

In [ ]:
path_train = Path("./training_set.pickle")
path_test = Path("./test_set.pickle")

with open(path_train, "rb") as f:
    training_set = pickle.load(f)
with open(path_test, "rb") as f:
    test_set = pickle.load(f)

sequences_negatives = set(training_set["negative"] + test_set["negative"])

In [ ]:
pad_token = "P"
seq_len = 151 #even number

path = Path("./gene_results_imvp.pickle")
with open(path, "rb") as f:
    sequences_imvp = pickle.load(f)


for key in sequences_imvp.keys():
    sequence = sequences_imvp[key]["sequence"]
    mods = sequences_imvp[key]["mod_positions"]
    #pad both sides of the sequence
    len_pad = seq_len//2 + 1 
    sequence = pad_token*len_pad + sequence
    mods = [x+len_pad for x in mods]
    sequence = sequence + pad_token*len_pad

    sequences_imvp[key]["sequence"] = sequence
    sequences_imvp[key]["mod_positions"] = mods

all_sequences = []
for key in sequences_imvp.keys():
    sequence = sequences_imvp[key]["sequence"]
    mods = sequences_imvp[key]["mod_positions"]
    min_distances = sequences_imvp[key]["min_distances"]
    clusters = sequences_imvp[key]["clusters"]
    for i, mod in enumerate(mods):
        if seq_len%2 == 0:
            seq_now = sequence[mod-seq_len//2:mod+seq_len//2]
        else:
            seq_now = sequence[mod-seq_len//2:mod+seq_len//2+1]
                        
        dist_now = min_distances[i]
        if clusters[i] == "Type I":
            if dist_now > 15:
                all_sequences.append(seq_now)
        else:
            all_sequences.append(seq_now)


In [4]:
rng = random.Random(42)


keys_imvp = list(sequences_imvp.keys())
for_neg_impv = {}

for key in keys_imvp:
    sequence = sequences_imvp[key]["sequence"]
    mods = sequences_imvp[key]["mod_positions"]
    putative_Cs_positions = [i for i, x in enumerate(sequence) if x == "C" and i not in mods]
    for_neg_impv[key] = {"sequence": sequence, "mod_positions": mods, "putative_Cs_positions": copy.deepcopy(putative_Cs_positions)}

all_negatives = []

for key in for_neg_impv.keys():
    sequence = for_neg_impv[key]["sequence"]
    putative_Cs_positions = for_neg_impv[key]["putative_Cs_positions"]
    for i in putative_Cs_positions:
        if seq_len%2 == 0:
            all_negatives.append(sequence[i-seq_len//2:i+seq_len//2])
        else:
            all_negatives.append(sequence[i-seq_len//2:i+seq_len//2+1])

all_negatives = set(all_negatives)

In [10]:
#remove the negatives that are already in the training and test sets
all_negatives = all_negatives - sequences_negatives - set(all_sequences)
all_negatives = list(all_negatives)

In [ ]:
path_all_remaining_negatives = Path("./all_remaining_negatives.pickle")
with open(path_all_remaining_negatives, "wb") as f:
    pickle.dump(all_negatives, f)